In [1]:
import numpy as np
import gensim, logging
import csv, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

ImportError: No module named gensim

In [2]:
training_arr = np.load("training_arr.npy")
print(training_arr.shape)
training_arr_labels = np.load("training_arr_labels.npy")
print(training_arr_labels.shape)
testing_arr = np.load("test_arr.npy")
print(testing_arr.shape)
testing_arr_labels = np.load("test_arr_labels.npy")
print(testing_arr_labels.shape)

(317974, 1000)
(317974, 1)
(7215, 1000)
(7215, 1)


In [57]:
import random
def convert_label_to_vector(label):
    first_half, second_half = label.split(" ")
    return [ord(i) for i in first_half]
def data_generator():
    for label, vector in zip(training_arr_labels, training_arr):
        label_vectors = [convert_label_to_vector(z.strip().strip('\'\"')) for z in label[0].split(',')]
        left = np.vstack(label_vectors)
        right = np.vstack([vector]*len(label_vectors))
        for l, r in zip(left, right):
            ret = (np.hstack((r)).reshape((1, 1000)), np.hstack(l).reshape(1, 4))
            yield ret

def data_generator_test():
    n = 0
    for label, vector in zip(testing_arr_labels, testing_arr):
        label_vectors = [convert_label_to_vector(z.strip().strip('\'\"')) for z in label[0].split(',')]
        left = np.vstack(label_vectors)
        right = np.vstack([vector]*len(label_vectors))
        for l, r in zip(left, right):
            ret = (np.hstack(r).reshape((1, 1000)), np.hstack(l).reshape(1,4))
            yield ret

In [58]:
e = data_generator().next()
e

(array([[-0.0645229 ,  0.17206578,  0.27927458, -0.30743885,  0.13213426,
          0.17168853, -0.04445799,  0.10521594, -0.19461322, -0.07855076,
         -0.41951725, -0.09668623, -0.35180104,  0.01814128, -0.0385349 ,
         -0.08269156, -0.21654539,  0.26963288,  0.26394212, -0.0769471 ,
          0.08159757,  0.1462388 , -0.03166891,  0.02227308,  0.23033331,
          0.17491078, -0.15494405,  0.13411523, -0.37758556, -0.01522811,
         -0.06026023, -0.11446414, -0.1623046 ,  0.03186761,  0.21371846,
         -0.02242558,  0.16906044, -0.02776215,  0.00679719,  0.06852875,
          0.00765829,  0.02052856,  0.16629462,  0.02336159, -0.1593455 ,
          0.01478121,  0.07531595,  0.01106454,  0.08919558,  0.09136292,
         -0.08452553, -0.07023416,  0.13372375, -0.06224204,  0.19001652,
         -0.05064945, -0.20634146, -0.3096862 , -0.04973361, -0.01662633,
         -0.06698404,  0.03559001,  0.19376872,  0.00359708,  0.18056893,
          0.09578311,  0.06466607,  0.

In [60]:
import tensorflow as tf
import os
#tf.disable_eager_execution()
dataset = tf.data.Dataset.from_generator(data_generator, (tf.int64, tf.int64), (tf.TensorShape([1, 1000]), tf.TensorShape([1, 4])))
test_dataset = tf.data.Dataset.from_generator(data_generator_test, (tf.int64, tf.int64), (tf.TensorShape([1, 1000]), tf.TensorShape([1, 4])))
test_dataset = test_dataset.batch(128, drop_remainder=True).repeat()
train_dataset = dataset.batch(32, drop_remainder=True).repeat()
print(train_dataset)
print(test_dataset)
for x, y in train_dataset:
    print(x, y)

<RepeatDataset shapes: ((32, 1, 1000), (32, 1, 4)), types: (tf.int64, tf.int64)>
<RepeatDataset shapes: ((128, 1, 1000), (128, 1, 4)), types: (tf.int64, tf.int64)>


RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.

In [61]:
#train_dataset = tf.data.Dataset.zip((tf.data.Dataset.from_tensors((training_vectors[:300000], training_labels[:300000])), tf.data.Dataset.from_tensors((training_vectors[300000:], training_labels[300000:]))))
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, input_shape=(None, 1000)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu')
])
saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)
model.summary()
model.compile(loss='mse',
              optimizer=tf.train.AdamOptimizer(1e-4),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, None, 1000)        1001000   
_________________________________________________________________
dense_39 (Dense)             (None, None, 100)         100100    
_________________________________________________________________
dense_40 (Dense)             (None, None, 4)           404       
Total params: 1,101,504
Trainable params: 1,101,504
Non-trainable params: 0
_________________________________________________________________


In [62]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 verbose=1)
history = model.fit(train_dataset, epochs=1,
                    validation_data=test_dataset, 
                    validation_steps=600,
                   steps_per_epoch=4000)



Epoch 1/1
4000/4000 [==============================] - 1356s 339ms/step - loss: 1560.5498 - acc: 0.5962 - val_loss: 1287.2079 - val_acc: 0.6595


In [63]:
test_loss, test_acc = model.evaluate(test_dataset, steps=100)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

100/100 [==============================] - 82s 818ms/step
Test Loss: 1328.19716919
Test Accuracy: 0.89375


In [64]:
from numpy import newaxis
p = model.predict(np.vstack(testing_arr).reshape(-1, 1, 1000))



In [69]:
for label, vector in zip(testing_arr_labels, testing_arr):
    label_vectors = [convert_label_to_vector(z.strip().strip('\'\"')) for z in label[0].split(',')]
    print(label_vectors[0])
    p = model.predict(np.vstack(vector).reshape(1, -1, 1000))
    print([chr(i) for i in p[0]])

[65, 48, 49, 72]


TypeError: only size-1 arrays can be converted to Python scalars